# การทำนายความยาวกลีบเลี้ยงด้วยชุดข้อมูลดอกไอริส

ในคาบนี้ จะใช้ชุดข้อมูลดอกไอริสจากคาบที่ผ่านมา ในการสร้างโมเดล Linear regression เพื่อทำนายความยาวกลีบเลี้ยง (sepal length) โดยจุดประสงค์การเรียนรู้ มีดังนี้

1. สามารถแบ่งข้อมูลเป็น training set และ test set ได้ และเข้าใจว่าแบ่งไปทำไม
2. เข้าใจที่มาของปัญหา Linear regression
3. ฝึกสอนโมเดล Linear regression ได้
4. ใช้โมเดลที่ฝึกสอนแล้วมาทำนายได้
5. ประเมินประสิทธิภาพของโมเดล Linear regression ไดุ้
6. สามารถทำ Standardization/Normalization กับข้อมูลได้

In [6]:
%reset -f
import pandas as pd
import plotly.express as px

## Load data

In [7]:
!wget -c "https://archive.ics.uci.edu/static/public/53/iris.zip" -O iris.zip
!unzip -o iris.zip -d iris_data

--2024-01-21 21:13:03--  https://archive.ics.uci.edu/static/public/53/iris.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘iris.zip’

iris.zip                [ <=>                ]   3.65K  --.-KB/s    in 0s      

2024-01-21 21:13:05 (193 MB/s) - ‘iris.zip’ saved [3738]

Archive:  iris.zip
  inflating: iris_data/Index         
  inflating: iris_data/bezdekIris.data  
  inflating: iris_data/iris.data     
  inflating: iris_data/iris.names    


หลังจากดาวน์โหลดและแตกไฟล์ zip แล้ว เราทำการอ่านไฟล์ด้วย `pd.read_csv()` เหมือนครั้งที่ผ่านมา และถือโอกาสกำหนดชื่อของคอลัมน์ไปด้วยโดยกำหนดพารามิเตอร์ `names=` และชนิดข้อมูลของแต่ละคอลัมน์ด้วย `dtype=`

In [8]:
import pandas as pd
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
column_types = {'sepal_length': 'float64',
                'sepal_width': 'float64',
                'petal_length': 'float64',
                'petal_width': 'float64',
                'class': 'string'}
iris = pd.read_csv('iris_data/iris.data',
                   header=None,
                   names=column_names,
                   dtype=column_types)
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    string 
dtypes: float64(4), string(1)
memory usage: 6.0 KB


จากนั้นลองวาด scatter matrix

In [9]:
import plotly.express as px

# df.select_dtypes() เลือกเฉพาะคอลัมน์ตามชนิดตัวแปรที่ต้องการ ซึ่งในกรณีนี้เราจะเอาแค่ float64
iris_number_only = iris.select_dtypes(include='float64')
px.scatter_matrix(iris_number_only)

## โมเดลเชิงเส้นที่รับอินพุต 1 คุณลักษณะ (Simple linear regression)

จากกราฟ เป็นที่น่าสนใจว่า น่าจะสามารถใช้ petal width ในการทำนาย sepal length ได้ แบบไม่ต้องสนใจสายพันธ์ของดอกไอริสเลย โดยใช้เพียง**สมการหรือฟังก์ชันเชิงเส้น (Linear function)** เท่านั้น เพราะจากการดูด้วยตาเปล่า แนวโน้มของค่า (trendline) จะเป็นเส้นตรง

> **Note:** การวิเคราะห์ข้อมูลในรูปแบบที่ต้องการทำนายค่าอะไรสักอย่างนั้น จะเรียกว่า **Predictive data analytics** ซึ่งแตกต่างจากเนื้อหาในคาบที่ผ่านมาที่เรียกว่า **Descriptive data analytics** (หรือ Exploratory Data Analysis (EDA)) ที่มุ่งแต่การดูแนวโน้มและข้อสรุปจากข้อมูลในอดีต 

การที่นำค่า **petal width** หนึ่งค่า มาทำนาย **sepal length** นั้น เรากล่าวว่า

- **petal width** เป็นตัวแปรต้น (Independent variable) แทนด้วยตัวแปร $x$ หรือในสาขาการวิเคราะห์ข้อมูล มักเรียก $x$ ว่าเป็น **คุณลักษณะ (หรือ feature)**
- **sepal length** เป็นตัวแปรตาม (Dependent variable) แทนด้วยตัวแปร $y$  ซึ่งในสาขาการวิเคราะห์ข้อมูล มักเรียก $y$ ว่าเป็น **เป้าหมายของการทำนาย (หรือ target)**

จากการดูด้วยตาเปล่าที่ว่า ความสัมพันธ์ระหว่าง petal width และ sepal length เป็นความสัมพันธ์เชิงเส้น (linear) นั้น สำหรับในส่วนของ Plotly จะมีความสามารถในการเพิ่มเส้นแนวโน้มเส้นตรง (trendline) ให้กับกราฟ `px.scatter()` ได้ โดยการกำหนดค่า `trendline='ols'` ดังนี้




In [10]:
fig = px.scatter(iris, x='petal_width',
           y='sepal_length',
           color='class',
           opacity=0.65,   # ความโปร่งแสงของแต่ละจุดข้อมูล เพื่อที่จะได้เห็นจุดที่ซ้อนกันเข้มกว่าจุดอื่น ๆ
           trendline='ols',   # ols = Ordinary Least Square เป็นการหาเส้นตรงที่ fit กับจุดข้อมูลที่สุดมาแสดง
           trendline_scope='overall', # คำนวนเส้นตรงจากข้อมูลทุกจุด ไม่แยกเป็นตาม color (ถ้าไม่กำหนดแล้วจะเกิดอะไรขึ้น?)
           trendline_color_override='darkblue')  # สีของเส้น trendline
fig.show()

ModuleNotFoundError: No module named 'statsmodels'

>**Note:**
>- OLS ย่อมาจาก Ordinary Least Squares เป็นรูปแบบการคำนวณหาคำตอบของ Linear regression แบบหนึ่ง
>- ศึกษาเพิ่มเติมการสร้าง trendline ใน plotly ได้ที่ https://plotly.com/python/linear-fits/



สมการ trendline ข้างต้น มีสมการคือ $f(x) = \hat{y} = w x + b$ โดยที่ $w$ และ $b$ เป็นสัมประสิทธิ์ของสมการ ที่มีความหมายคือ
- $w$ คือ ความชัน (slope)
- $b$ คือ ค่าตัดแกนตั้ง (offset)

เราให้ $\hat{y}$ แทนเอาต์พุตของสมการ trendline เพื่อหมายถึงการเป็นค่าประมาณของ $y$ กล่าวคือ เมื่อป้อน $x$ ค่าหนึ่ง ๆ เข้าสมการ $f(x) = w x + b$ แล้ว ค่าที่ได้อาจจะไม่เท่ากับค่า $y$ จริง ๆ ก็ได้


เราเรียกโมเดลที่ใช้ฟังก์ชันเชิงเส้นในการทำนายค่าที่เป็นจำนวนจริงใด ๆ จะว่าเป็นโมเดล **Linear regression** ซึ่งปัญหาของ Linear regression ก็คือการหาค่า $w$ และ $b$ ที่ดีที่สุด (ใกล้เคียงข้อมูลที่สุด, fit ข้อมูลมากที่สุด) แล้วเราจะหาได้อย่างไร?

พิจารณาตัวอย่างข้อมูลเฉพาะ `petal_width` กับ `sepal_length` และผลการทำนายที่ติดตัวแปรไม่ทราบค่า $w$ และ $b$ ต่อไปนี้

| แถวที่ | ค่า petal_width ($x$) | ค่า sepal_length จริง ๆ ($y$) | $\rightarrow$ |sepal_length ที่ทำนายได้ ($\hat{y} = wx+b$)  |
|:--|:--|:--|:-- |:-- |
| 30 |	$x^{(30)} = 0.2$ |	$y^{(30)} = 4.8$ |$\rightarrow$ | $\hat{y}^{(30)} =  0.2 w + b$ |
| 84	| $x^{(84)} = 1.5$	| $y^{(84)} = 5.4$ |$\rightarrow$ |  $\hat{y}^{(84)} = 1.5w + b$ |
| 128 |	$x^{(128)} = 2.1$	| $y^{(128)} = 6.4$ |$\rightarrow$ | $\hat{y}^{(128)} = 2.1 w + b$  |
| 134	| $x^{(134)} = 1.4$	| $y^{(134)} = 6.1$ |$\rightarrow$ | $\hat{y}^{(134)} = 1.4 w + b$  |
| 145	| $x^{(145)} = 2.3$	| $y^{(145)} = 6.7$ |$\rightarrow$ |  $\hat{y}^{(145)} = 2.3 w + b$ |

โดยที่ $x^{(i)}$ และ $y^{(i)}$ คือค่าของแถวที่ $i$

จากตาราง จะสามารถประเมินความผิดพลาดในการทำนายข้อมูลแต่ละแถว ได้ดังนี้

| แถวที่ |  ความผิดพลาดของการทำนาย (Error) |
|:--|:--  |
| 30 | $\hat{y}^{(30)}- y^{(30)} = 0.2 w + b - 4.8$ |
| 84| $\hat{y}^{(84)}- y^{(84)} = 1.5w + b - 5.4$ |
| 128| $\hat{y}^{(128)}- y^{(128)} = 2.1 w + b - 6.4$ |
| 134| $\hat{y}^{(134)}- y^{(134)} = 1.4 w + b - 6.1$|
| 145 | $\hat{y}^{(145)}- y^{(145)} = 2.3 w + b - 6.7$|

> **Note:** ในทางสถิติมักเรียก Error ว่า Residual ซึ่งแม้ว่าทั้งสองคำนี้จะมีความแตกต่างกันอยู่บ้าง แต่ในวิชานี้จะใช้สองคำนี้แทนกัน ซึ่งหมายถึงความผิดพลาดในการทำนาย 

ค่าความผิดพลาดข้างต้น เป็นได้ทั้งค่าบวกและลบ ดังนั้นการจะนำความผิดพลาดรวมกัน จึงจะทำการยกกำลังสองเสียก่อน แล้วจึงค่อยมาบวกกัน ซึ่งการหาค่าความผิดพลาดลักษณะนี้ เรียกว่า  **Mean Squared Error หรือ MSE**

จะได้ว่า สำหรับดอกไอริส 5 ดอกนี้ ทำนายผิดพลาดโดยเฉลี่ย คือ

$
MSE = \frac{(0.2 w + b - 4.8)^2 + (1.5w + b - 5.4)^2 + (2.1 w + b - 6.4)^2 + (1.4 w + b - 6.1)^2 + (2.3 w + b - 6.7)^2}{5}
$

หรือสุดท้ายแล้ว หากกระจายพหุนามแล้ว จะได้

$
MSE = 2.79 w^2 + b^2 + 3 wb- 18.58 w - 11.76 b  + 35.052   
$

การไม่ทราบค่า $w$ และ $b$ จะทำให้ค่า MSE ติดอยู่ในรูปตัวแปร 2 ตัวนี้ ซึ่งสำหรับปัญหา Linear regression แล้วนั้น จะเป็นการเลือก $w$ และ $b$ ที่ให้ค่า MSE ต่ำที่สุด

>**Note:** จะใช้ค่าสัมบูรณ์แทนการยกกำลังสองก็ได้ แต่ในทางคณิตศาสตร์ การยกกำลังสองจะสะดวกในการหาค่า $w$ และ $b$ ที่ดีที่สุดมากกว่า


สำหรับกรณีทั่วไปที่มีดอกไอริส $n$ ดอก สูตรคำนวณ MSE จะเขียนได้ ดังนี้

$
MSE = \frac{1}{n} \sum_{i=1}^n (\hat{y}^{(i)} - y^{(i)})^2
$

โดยที่ $\hat{y}^{(i)} = w x^{(i)} + b$

ซึ่งปัญหา Linear regression ก็คือปัญหาของการเลือก $w$ และ $b$ ของ $\hat{y}^{(i)} = w x^{(i)} + b$ ที่ให้ค่า MSE ต่ำที่สุด โดยใช้ข้อมูล $n$ คู่ คือ ($x^{(i)},y^{(i)}$), $i=1,2,...,n$ ในการเลือก และโดยทั่วไป มักเขียนด้วยสัญลักษณ์ทางคณิตศาสตร์ ดังต่อไปนี้

$
\min_{w,b} \frac{1}{n} \sum_{i=1}^n (\hat{y}^{(i)} - y^{(i)})^2
$

> **Note:**
> สำหรับในคาบนี้ จะไม่สนใจว่าวิธีการทางคณิตศาสตร์จะแก้ปัญหาข้างต้นเพื่อหา $w$ และ $b$ ได้อย่างไร แต่อยากให้นักศึกษาทำความเข้าใจกราฟรูปนี้ แล้วในรูป จะเห็นว่า สุดท้ายแล้ว Linear regression คือการหาสัมประสิทธิ์ของสมการเส้นตรง (เส้นสีฟ้า) ที่ทำให้ผลรวมของความยาวของเส้นแนวตั้งทุกเส้น (เส้นสีเขียว) โดยรวมให้สั้นที่สุด
>
> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Linear_least_squares_example2.svg/782px-Linear_least_squares_example2.svg.png" height="200">




### Data splits ด้วย `train_test_split()`

ข้อมูลดอกไอริสมีทั้งหมด 150 แถว (3 สายพันธุ์ สายพันธุ์ละ 50 แถว) การจะนำข้อมูล*ทั้งหมด*มาสร้างโมเดล (หรือก็คือมาใช้หาฟังก์ชัน $\hat{y} = wx+b$) ก็สามารถทำได้ แต่หากทำแล้ว เราก็จะไม่มีข้อมูลเหลือไว้สำหรับประเมินประสิทธิภาพของโมเดล (Model evaluation) แบบไม่ลำเอียง และถ้าเราวัดประสิทธิภาพแบบไม่ลำเอียงไม่ได้ ก็ไม่มีเหตุผลที่จะสร้างโมเดลแต่แรก

ดังนั้น ในทางปฏิบัติ มักจำเป็นต้องแบ่งข้อมูลเป็น 2 กลุ่ม ดังนี้
- **กลุ่มที่ 1** ไว้สำหรับสร้างโมเดล (หาพารามิเตอร์ $w$ และ $b$ ของโมเดล) เราจะเรียกข้อมูลกลุ่มนี้ว่า **ชุดข้อมูลฝึกสอน/เทรน (training set)**
- **กลุ่มที่ 2** ไว้สำหรับทดสอบโมเดล เราเรียกชุดข้อมูลกลุ่มนี้ว่า **ชุดข้อมูลทดสอบ (test set)** ซึ่งชุดข้อมูลทดสอบนี้ จะไม่ได้มีส่วนในการหาพารามิเตอร์ $w$ และ $b$ ของโมเดลเลย ดังนั้นบางครั้งจึงถูกเรียกว่า blind test set

การแบ่งข้อมูล สามารถทำได้ง่ายโดยใช้ฟังก์ชัน `train_test_split()` ของ `scikit-learn` ตามรูปแบบดังนี้

ขั้นแรก คือนำเข้าคำสั่ง
```python
from sklearn.model_selection import train_test_split
```

โดยรูปแบบการ split ข้อมูลแบบง่ายที่สุดคือ
```python
df1, df2 = train_test_split(df, train_size=สัดส่วนการแบ่ง, random_state=None)
```
ซึ่งจะทำการสุ่มเลือกแถวของ `df` เป็น 2 กลุ่ม และเก็บไว้ในตัวแปร `df1` และ `df2`

> **Note:** `random_state=` ใช้สำหรับกำหนดตัวเลข (ค่า seed) เพื่อล็อกผลการสุ่ม เช่น `random_state=555` ซึ่งในตัวอย่างต่อ ๆ ไป จะทำการล็อกค่าไว้เพื่อให้ผลลัพธ์ที่นักศึกษาได้ ตรงกับของอาจารย์

ยกตัวอย่างเช่น หากต้องการแบ่ง 60% สำหรับเทรน และ 40% สำหรับเทส จะทำได้ดังนี้

In [ ]:
from sklearn.model_selection import train_test_split
iris_train, iris_test = train_test_split(iris,
                                         train_size=0.6, # ต้องการ training set ดอกมี 60% ของ 150 ดอก
                                         random_state=123)
print(f'Shape of training set: {iris_train.shape}')
print(f'Shape of test set: {iris_test.shape}')

Shape of training set: (90, 5)
Shape of test set: (60, 5)


จากผลข้างต้น ลองมาดูกันว่ามีสายพันธุ์อะไรบ้างอยู่ใน training set สายพันธุ์ละกี่แถว?

In [ ]:
iris_train['class'].value_counts(normalize=True) # กำหนด normalize เพื่อให้ได้ค่าสัดส่วน

Iris-versicolor    0.377778
Iris-setosa        0.311111
Iris-virginica     0.311111
Name: class, dtype: Float64

เนื่องจากการสุ่มไม่ได้สนใจเลยว่า training set (`iris_train`) และ test set (`iris_test`) ที่ได้ จะมีสัดส่วนของสายพันธุ์ดอกไอริสที่เท่า ๆ กับข้อมูลตั้งต้น (`iris`) หรือไม่ ซึ่งในกรณีนี้ จะเห็นว่า หลังจากที่แบ่ง train/test ด้วยสัดส่วน 80/20 แล้ว เราได้ ข้อมูลเทรนที่มีสัดส่วนสายพันธุ์เป็น Versicolor/Virginica/Setosa = 36.6/32.5/30.8

ในหลายกรณี สัดส่วนที่ไม่เท่ากันข้างต้นก็ไม่ได้เป็นผลเสียแต่อย่างใด แต่ถ้าหากต้องการแบ่งให้คำนึงถึงสัดส่วนของสายพันธุ์ด้วยนั้น  ก็สามารถทำได้โดยกำหนด `stratify=รายการชนิดสายพันธ์` ดังนี้

In [ ]:
labels = iris['class']  # เลือกคอลัมน์ของสายพันธุ์
iris_train, iris_test = train_test_split(iris,
                                         train_size=0.6,
                                         stratify=labels,
                                         random_state=123)
iris_train['class'].value_counts(normalize=True) # กำหนด normalize เพื่อให้ได้ค่าสัดส่วน

Iris-setosa        0.333333
Iris-virginica     0.333333
Iris-versicolor    0.333333
Name: class, dtype: Float64

จะเห็นว่า หลังจากที่กำหนดพารามิเตอร์ `stratify=` แล้ว จะได้ข้อมูลที่มีสัดส่วนสายพันธุ์เท่ากับสัดส่วนในข้อมูลต้นฉบับ

**คำถาม:** ทำไมจึงอยากที่จะให้สัดส่วนสายพันธุ์ในข้อมูลหลังแบ่ง ให้เหมือนกับสัดส่วนสายพันธุ์ในข้อมูลก่อนแบ่ง?

นอกจากนี้ คำสั่ง `train_test_split()` ยังสามารถแบ่งข้อมูลในกรณีที่ข้อมูลอินพุต(ตัวแปรต้น) กับข้อมูลเอาต์พุต (ตัวแปรตาม) ถูกแยกเก็บไว้คนละตัวแปร ได้อีกด้วย ดังตัวอย่างต่อไปนี้

In [ ]:
from sklearn.model_selection import train_test_split
X = iris[['petal_width']]  # X เป็น DataFrame
Y = iris[['sepal_length']] # Y เป็น DataFrame
label = iris[['class']]    # label เป็น DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.6,
                                                    stratify=labels,
                                                    random_state=123)
print(f'Shape of training set: X_train {X_train.shape}, y_train {y_train.shape}')
print(f'Shape of test set:     X_test  {X_test.shape},  y_test  {y_test.shape}')

Shape of training set: X_train (90, 1), y_train (90, 1)
Shape of test set:     X_test  (60, 1),  y_test  (60, 1)


จากโค้ดข้างต้น เราแยกตัวแปรต้นและตัวแปรตาม เก็บไว้ในตัวแปร `X` และ `Y` ตามลำดับ ซึ่งในการ split นั้น คำสั่ง `train_test_split()` จะคำนึงถึงแถวของ `X` และ `Y` คู่กัน และให้ถูกเลือกไปด้วยกันตอน split ซึ่งผลลัพธ์ของการ split จะได้ DataFrame อยู่ 4 ตัวแปร

### การฝึกสอนโมเดลด้วย `model.fit()`

เมื่อมีชุดข้อมูลพร้อมแล้วสำหรับเทรน ขั้นตอนต่อไปจะเป็นการฝึกสอนโมเดลให้เรียนรู้ชดข้อมูลนั้น

ซึ่งในขั้นแรก จะต้องสร้างโมเดลตั้งต้นมาก่อน ซึ่งจะเป็นโมเดลที่จะยังไม่ได้เรียนรู้ ซึ่งสร้างได้ ดังนี้








In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model

LinearRegression()

ในกรณีนี้ โมเดลถูกเก็บในตัวแปร `model` ซึ่งการฝึกสอนโมเดล ทำได้โดยใช้คำสั่ง `model.fit()` ซึ่งมีรูปแบบการใช้งาน ดังนี้

```python
model.fit(ตัวแปรต้น,ตัวแปรตาม)
```

In [ ]:
model.fit(X_train, y_train)

LinearRegression()

<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-predictor.fit.svg" width=50%>
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

โดยหลังจาก `model.fit()` แล้ว จะสามารถเข้าถึงสัมประสิทธิ์ของสมการ $\hat{y} = wx+b$ ได้จาก
- `model.coef_` ซึ่งจะได้ค่า $w$
- `model.intercept_` ซึ่งจะได้ค่า $b$

In [ ]:
w = model.coef_
b = model.intercept_
print(f'สมการคือ f(x) = {w}*x + {b}')

สมการคือ f(x) = [[0.85559634]]*x + [4.7620008]


### การทำนายด้วย `model.predict()`

การจะนำโมเดลนี้ไปใช้ทำนาย ก็สามารถทำได้โดยการป้อนค่า petal_width ($x$) ทีละแถวเข้าไป ก็จะได้ผลการทำนายค่า sepal_length ($\hat{y}$) ซึ่งอาจคำนวณโดยใช้ Pandas ดังนี้

In [ ]:
0.85559634*X_train + 4.7620008

,petal_width
24,4.933120
147,6.473193
88,5.874276
123,6.302074
31,5.104239
...,...
52,6.045395
41,5.018680
12,4.847560
20,4.933120


> **Note:** ผลลัพธ์ข้างต้นแสดงคอลัมน์ชื่อ `petal_width` แต่จริง ๆ คือค่า sepal length ที่ทำนายได้ของแต่ละดอก

โค้ดข้างต้นเป็นการลองให้โมเดลทำนายดอกใน training set ซึ่งในกรณีต้องการให้ทำนายดอกใน test set ก็ทำลักษณะเดียวกัน แค่เปลี่ยนตัวแปรต้น ดังนี้

```python
0.85559634*X_test + 4.7620008
```



อย่างไรก็ตาม มีวิธีการให้โมเดลทำนาย แบบที่สะดวกกว่าโค้ดข้างต้น โดยการเรียกใช้ฟังก์ชัน `model.predit()`


<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-predictor.predict.svg" width=50%>
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

In [ ]:
y_train_predict = model.predict(X_train)  # ทำนาย training set ให้ผลลัพธ์เหมือนการคำนวณจากสมการโดยตรง
y_train_predict

array([[4.93312007],
       [6.47319349],
       [5.87427605],
       [6.30207422],
       [5.10423934],
       [6.04539531],
       [4.93312007],
       [5.87427605],
       [4.93312007],
       [4.93312007],
       [5.87427605],
       [6.30207422],
       [5.0186797 ],
       [6.04539531],
       [6.30207422],
       [5.0186797 ],
       [6.72987239],
       [5.95983568],
       [4.93312007],
       [4.93312007],
       [5.78871641],
       [6.72987239],
       [6.55875312],
       [6.72987239],
       [6.30207422],
       [5.10423934],
       [6.64431275],
       [5.87427605],
       [5.95983568],
       [5.87427605],
       [5.70315678],
       [5.87427605],
       [5.61759714],
       [5.87427605],
       [6.81543202],
       [5.78871641],
       [6.55875312],
       [4.93312007],
       [6.04539531],
       [6.55875312],
       [5.87427605],
       [5.78871641],
       [6.47319349],
       [4.93312007],
       [4.93312007],
       [5.0186797 ],
       [5.78871641],
       [5.874

คำสั่ง `model.predict()` ให้ผลลัพธ์เป็นตัวแปรชนิด Numpy array ซึ่งไม่ใช่ DataFrame/Series ของ Pandas ซึ่งแม้ว่า Numpy จะเป็นไลบรารีในที่เป็นที่นิยมมาก และแทบจะขาดไม่ได้ในการคำนวณทางคณิตศาสตร์ แต่เนื่องจากเวลาของวิชานี้มีจำกัด ดังนั้นวิชานี้จะไม่ลงรายละเอียดการใช้งาน Numpy ซึ่งหากนักศึกษาสนใจ อาจศึกษาเพิ่มเติมจาก https://numpy.org/ หรือจากแหล่งอื่น ๆ

In [ ]:
type(y_train_predict)   # จะเห็นว่าไม่ใช่ชนิด DataFrame

numpy.ndarray

 เราอาจลองสร้าง DataFrame ขึ้นมาอันใหม่ไว้  รวบรวมผลการทำนาย (สมมติว่าไม่อยากแตะต้องข้อมูลต้นฉบับ, อันนี้เป็นตัวอย่างเฉย ๆ เพราะมันทำได้หลายแบบ)






In [ ]:
# สร้าง copy ใหม่ของ X_train เพราะไม่ต้องการยุ่งกับ X_train ต้นฉบับ เพื่อป้องกันกรณีที่เผลอแก้ค่าใน comparison แล้วค่าใน X_train เปลี่ยนตาม
comparison = X_train.copy()

# เพิ่มคอลัมน์ sepal length ที่เป็นค่าตัวแปรตามที่แท้จริงเข้าไป
# Note: ทำได้ เพราะ X_train กับ y_train มี index เหมือนกันเป๊ะ
comparison['sepal_length'] = y_train

# เพิ่มคอลัมน์ผลการทำนาย
# Note: ทำได้ไม่ผิด เพราะแม้ว่า y_train_predict จะเป็น Numpy array ที่ไม่มี index เดิมของ y_train
#       แต่ตอนทำนายด้วย model.predict() นั้น ไม่ได้มีการสลับลำดับแถว
comparison['sepal_length_predict'] = y_train_predict

comparison

,petal_width,sepal_length,sepal_length_predict
24,0.2,4.8,4.933120
147,2.0,6.5,6.473193
88,1.3,5.6,5.874276
123,1.8,6.3,6.302074
31,0.4,5.4,5.104239
...,...,...,...
52,1.5,6.9,6.045395
41,0.3,4.5,5.018680
12,0.1,4.8,4.847560
20,0.2,5.4,4.933120


จาก DataFrame `comparison` เราสามารถใช้ ploty express ในการวาดกราฟเปรียบเทียบได้โดยง่าย

In [ ]:
px.scatter(comparison, x='petal_width',
           y=['sepal_length','sepal_length_predict'],
           opacity=0.65)

**ลองทำ:** ลองเพิ่ม `trendline='ols'` เข้าไปดูสิ
```python
px.scatter(comparison, x='petal_width',
           y=['sepal_length','sepal_length_predict'],  
           trendline='ols',  # เพิ่มเข้ามา
           opacity=0.65)
```           

จะเห็นว่าผลการทำนายจาก `model.predict()` อยู่บนเส้นเดียวกับเส้น trendline ที่ Plotly สร้างมาให้เลย แปลว่าเรามาถูกทาง!

### การคำนวณ MSE ด้วย `mean_squared_error()`

การใช้ `model.predict()` เป็นเพียงการทำนายค่า sepal length ออกมา ซึ่งการจะประเมินว่าทำนายดีแค่ไหนนั้นดูยาก เพราะต้องดูเทียบค่าความผิดพลาดในการทำนายเป็นรายดอก อย่างเช่น

In [ ]:
# คำนวณผลต่างของการทำนาย ยกกำลังสอง
comparison['Error^2'] = (comparison['sepal_length'] - comparison['sepal_length_predict'])**2  # Error ยกกำลังสอง
comparison.head()

,petal_width,sepal_length,sepal_length_predict,Error^2
24,0.2,4.8,4.933120,0.017721
147,2.0,6.5,6.473193,0.000719
88,1.3,5.6,5.874276,0.075227
123,1.8,6.3,6.302074,0.000004
31,0.4,5.4,5.104239,0.087474


โดยปกติแล้ว ในการประเมินประสิทธิภาพของโมเดล ในขั้นแรกเราจะยังไม่สนใจประสิทธิภาพการทำนายเป็นรายดอก แต่จะสนใจประสิทธิภาพของโมเดลโดยรวมเป็นอันดับแรก ซึ่งกรณีของ Linear regression ก็นิยมใช้ MSE ในการประเมิน อันที่จริง จากตัวแปร `comparison` เราสามารถคำนวณค่า MSE ของ training set ได้เองแล้ว ดังนี้

In [ ]:
mse = comparison['Error^2'].mean()
print(f'MSE จากการคำนวณเอง = {mse}')

MSE จากการคำนวณเอง = 0.2048398868775438


> **Note:** เนื่องจาก MSE เป็นการนำ Error มายกกำลังสอง ดังนั้นในกรณีของ MSE = 0.2048 จึงมีหน่วย $cm^2$ เพราะ sepal length มีหน่วย $cm$

อย่างไรก็ตาม Scikit-learn ก็ได้อำนวยความสะดวกในการคำนวณ MSE เช่นกันด้วยฟังก์ชัน `mean_squared_error()` ที่ต้องทำการนำเข้าฟังก์ชันก่อนการใช้งาน ดังนี้

```python
from sklearn.metrics import mean_squared_error
```
ซึ่งในการเรียกใช้ จะเป็นดังนี้
```python
mean_squared_error(ค่าที่แท้จริง, ค่าที่ทำนายได้)
```

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, y_train_predict)
print(f'MSE จาก mean_squared_error() = {mse}')

MSE จาก mean_squared_error() = 0.2048398868775438


### MSE เปรียบเทียบระหว่าง Training set กับ Test set

พิจารณาโค้ดต่อไปนี้ ที่นำโมเดลที่ได้เรียนรู้ training set มาใช้ทำนายข้อมูล training set และ test set เปรียบเทียบกัน

In [ ]:
# ประสิทธิภาพกรณีทำนาย training set
y_train_pred = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)

# ประสิทธิภาพกรณีทำนาย test set
y_test_pred = model.predict(X_test)
mse_test  = mean_squared_error(y_test, y_test_pred)

print(f'MSE จากการทำนาย training set = {mse_train:.3f}') # เพิ่ม :.3f เข้าไปเพื่อพิมพ์ทศนิยมแค่ 3 ตำแหน่ง
print(f'MSE จากการทำนาย test set     = {mse_test:.3f}')

MSE จากการทำนาย training set = 0.205
MSE จากการทำนาย test set     = 0.266


จากโค้ดข้างต้น จะเห็นว่า MSE ของ test set มีค่าสูงกว่าของ training set นักศึกษาคิดเห็นอย่างไร จงอภิปราย
1. ถ้าโมเดลทำนายดี ค่า MSE ควรมากหรือน้อย
2. ค่า MSE ต่ำที่สุดที่เป็นไปได้คือเท่าไร
3. การที่ MSE ของ test set สูงกว่าของ training set นั้น สมเหตุสมผลหรือไม่

### (ศึกษาเอง) การวาดกราฟโดยใช้ `plotly.graph_objects`



`plotly.express` เป็นไลบรารีการวาดกราฟที่สร้างขึ้นเพื่ออำนวยความสะดวกในการวาดกราฟจาก DataFrame ดังนั้นจึงมีข้อจำกัดบางประการในการสร้างกราฟ ดังนั้นในหัวข้อนี้ จะนำเสนอการวาดกราฟที่ควบคุมรายละเอียดได้มากขึ้น โดยการใช้ `plotly.graph_objects` [(ดูเพิ่ม)](https://plotly.com/python/graph-objects/)ซึ่งแม้ว่า `plotly.graph_objects` จะสามารถสร้างกราฟ เริ่มต้นจากกราฟเปล่า ๆ ได้ (ใช้ `go.Figure()` แล้วค่อยเพิ่มเติมสิ่งต่าง ๆ เข้าไปในกราฟได้

> **Note:**  ในทางปฏิบัติ แนะนำให้เริ่มต้นวาดกราฟโดยใช้ `plotly.express` ก่อน ซึ่งคำสั่งเช่น `fig = px.scatter()`, `fig = px.bar()`, `fig = px.hist()` เป็นต้น จะให้ตัวแปร `fig` ที่เป็นประเภท `plotly.graph_objects` ออกมา ซึ่งสามารถนำ `fig` ไปเพิ่มความซับซ้อนของกราฟในภายหลังได้


การนำเข้า `plotly.graph_objects` ทำดังนี้
```python
import plotly.graph_objects as go
```

ต่อไปนี้คือตัวอย่างการใช้ `plotly.graph_objects`

จากที่ผ่านมา เราใช้ `px.scatter()` วาดกราฟดอกไอริส 3 สายพันธ์พร้อมกันได้ในคำสั่งเดียว แต่ในตัวอย่างต่อไปนี้ จะลองว่า scatter plot ละสายพันธุ์เพิ่มเข้าไปในกราฟ

สมมติมีข้อมูลของ 3 สายพันธุ์ แยกเป็น DataFrame 3 อัน


In [ ]:
setosa = iris[iris['class'] == 'Iris-setosa']
virginica = iris[iris['class'] == 'Iris-virginica']
versicolor = iris[iris['class'] == 'Iris-versicolor']

**Step 1** `import` และสร้าง Figure เปล่า

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

**Step 2** สร้างวัตถุของกราฟ

เราต้องการใส่ scatter plot ของดอกแต่ละสายพันธุ์เข้าไปในกราฟ ดังนั้น จึงต้องสร้างวัตถุ Scatter ของแต่ละสายพันธุ์ก่อน โดยใช้ `go.Scatter()` [(อ้างอิง)](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter.html)



In [ ]:
object_1 =  go.Scatter(x=setosa['petal_width'],
                       y=setosa['sepal_length'],
                       mode='markers',
                       name='Setosa') # ของ Setosa
object_2 =  go.Scatter(x=virginica['petal_width'],
                       y=virginica['sepal_length'],
                       mode='markers',
                       name='Virginica') # ของ Virginica
object_3 =  go.Scatter(x=versicolor['petal_width'],
                       y=versicolor['sepal_length'],
                       mode='markers',
                       name='Versicolor') # ของ Versicolor

> ข้อสังเกตุ `go.Figure()` และ `go.Scatter()` ใช้สร้างวัตถุ Figure และวัตถุ Scatter ขึ้นมา แต่ยังไม่ได้เชื่อมหากัน

**Step 3** ใส่วัตถุของกราฟลงในกราฟ

ในขั้นนี้ จะใช้ `fig.add_trace()` ใช้เพื่อใส่กราฟเข้าไปใน Figure  

In [ ]:
fig.add_trace(object_1)
fig.add_trace(object_2)
fig.add_trace(object_3)

หรือหากจะวาดเส้นตรงเพิ่มเข้าไปในกราฟเอง ก็ย่อมทำได้ ก็ทำได้ในลักษณะเดียวกัน แต่คราวนี้ จะเขียนรวบในคำสั่งเดียว

สมมติต้องการเติมเส้นตรงที่ลากตามลำดับพิกัด `(0,6) -> (1.5,7.5)` ตามลำดับ จะเขียนโค้ดได้ดังนี้

In [ ]:
fig.add_trace(
      go.Scatter(x=[0, 1.5],
                 y=[6, 7.5],
                 mode="lines",
                 line=go.scatter.Line(color="gray"), # สีเส้น
                 showlegend=False  # ไม่ต้องแสดงชื่อเส้น
      )
)

**Step 4:** แก้ไขภาพรูปลักษณ์ด้านนอกของกราฟ

ใช้ `fig.update_layout()` ในการกำหนดค่าต่าง ๆ ดังตัวอย่างต่อไปนี้



In [ ]:
fig.update_layout(title_text="กราฟดอกไอริส", # ใส่ชื่อกราฟ
                  title_font_size=30,    # ขนาดฟอนต์ชื่อกราฟ
                  margin={'l':0, 'r':0, 't':60, 'b':0},  # ตัดขอบรูป l=left, r=right, t=top, b=bottom
)
fig.show()  # คำสั่งนี้ปกติจำเป็นต้องใช้เพื่อสั่งแสดงกราฟ เพียงแต่สำหรับ Google Colab โดยปกติ จะแสดงผลโดยอัตโนมัติ

เนื่องจากหัวข้อนี้เป็นเพียงการแสดงการใช้ `plotly.graph_objects` เพียงคร่าว ๆ  เท่านั้น ดังนั้น นักศึกษาสามารถดูเพิ่มเติมได้ที่
- การอัพเดทกราฟ https://plotly.com/python/creating-and-updating-figures/
- การปรับขนาดกราฟ https://plotly.com/python/setting-graph-size/ เป็นต้น

สำหรับในหัวข้อนี้ อย่างน้อยสิ่งที่ควรทราบเกี่ยวกับ `plotly.graph_objects` เบื้องต้นก็คือ `go.Figure()`, `go.Scatter()`, `fig.add_trace()`, และ `fig.update_layout()`


## โมเดลเชิงเส้นที่รับอินพุตมากกว่า 1 คุณลักษณะ (Multiple linear regression)

ในหัวข้อที่ผ่านมา เราได้ใช้ฟังก์ชันเชิงเส้นที่รับ `petal_width` มาเพื่อทำนายค่า `sepal_length` อย่างไรก็ตาม การใช้คุณลักษณะเดียวเป็นอินพุต อาจจะไม่เพียงพอที่จะทำนายค่า `sepal_lenth` ได้ดีนัก จึงอาจพิจารณาเพิ่มมิติข้อมูลเข้ามา

**<u>กรณี 2 คุณลักษณะ</u>**

สมมติว่าต้องการใช้ค่า **petal_width และ petal_length ในการทำนาย sepal_length** กรณีนี้

- **petal_width** เป็นตัวแปรต้นตัวที่ 1 หรือคุณลักษณะที่ 1 เขียนแทนด้วยตัวแปร $x_1$
- **petal_length** เป็นตัวแปรต้นตัวที่ 2 หรือคุณลักษณะที่ 2 เขียนแทนด้วยตัวแปร $x_2$
- **sepal length** เป็นตัวแปรตาม หรือเป้าหมายของการทำนาย แทนด้วยตัวแปร $y$  

การที่มีอินพุตเพิ่มมีอีก 1 มิติ สมการเส้นตรงจึงจะขยายกลายเป็นระนาบ (plane) ใน 3 มิติ แทน (มิติของ $x_1, x_2, \hat{y}$) ซึ่งมีสมการคือ

$$
f(x_1, x_2) = \hat{y} = w_1 x_1 + w_2 x_2 + b
$$

โดยที่ $w_1$, $w_2$ และ $b$ เป็นสัมประสิทธิ์ของสมการ เราอาจมองว่า $w_1$ และ $w_2$ เป็นตัวคูณถ่วงน้ำหนักของค่าคุณลักษณะก็ได้ ซึ่งหากวาดเป็นกราฟ ก็จะคือความชันของระนาบ

**<u>กรณี 3 คุณลักษณะ</u>**

สมมติว่าต้องการใช้ค่า **petal_width, petal_length และ sepal_width ในการทำนาย sepal_length** กรณีนี้ก็ไม่ต่างจากกรณี 2 คุณลักษณะเท่าใด เพียงแต่เพิ่มอีก 1 มิติเข้ามาคือ sepal_width จะเป็นตัวแปรต้นตัวที่ 3 ซึ่งเมื่อนำมาเขียนสมการ ก็จะได้สมการใน 4 มิติ (มิติของ $x_1, x_2, x_3, \hat{y}$) ซึ่งจะมีสมการคือ

$$
f(x_1, x_2, x_3) = \hat{y} = w_1 x_1 + w_2 x_2 + w_3 x_3 + b
$$
โดยที่ $w_1$, $w_2$, $w_3$ และ $b$ เป็นสัมประสิทธิ์ของสมการ

นักศึกษาลองคิดถึงกรณีทั่วไปดูว่า หากมี $d$ คุณลักษณะเป็นอินพุต ฟังก์ชันเชิงเส้นก็จะมีรูปสมการคือ

$$
f(x_1, x_2, ..., x_d) = \hat{y} = w_1 x_1 + w_2 x_2 + ... + w_d x_d + b
$$

กระนั้น แม้อินพุตจะมี $d$ คุณลักษณะ แต่เอาต์พุตของการทำนายยังเป็น $\hat{y}$ ค่าเดียว ดังนั้น การคำนวณ MSE ก็จะยังคงเหมือนเดิมกับกรณี 1 คุณลักษณะ

$$
MSE = \frac{1}{n} \sum_{i=1}^n (\hat{y}^{(i)} - y^{(i)})^2
$$
แต่ฟังก์ชันเชิงเส้นจะเปลี่ยนเป็น $\hat{y}^{(i)} = w_1 x_1^{(i)} + w_2 x_2^{(i)} + ... + w_d x_d^{(i)} + b$

ปัญหา Linear regression ในกรณนี้ จึงเป็นปัญหาของการเลือก $w_1, w_2, ..., w_d$ และ $b$ ที่ให้ค่า MSE ต่ำที่สุด ซึ่งมักเขียนด้วยสัญลักษณ์ทางคณิตศาสตร์ ดังนี้

$$
\min_{w_1, w_2, ..., w_n,b} \frac{1}{n} \sum_{i=1}^n (\hat{y}^{(i)} - y^{(i)})^2
$$

เราเรียก Linear regression ที่รับอินพุตหลายค่า ว่า **Multiple linear regression**

การแก้สร้างโมเดล Multiple linear regression ใน Scikit-learn ทำได้ในลักษณะเดียวกับกรณี Linear regression แบบธรรมดา (Simple linear regression) ที่ได้แสดงวิธีการไปแล้วก่อนหน้า

สำหรับในส่วนต่อไป จะทดลองทำ Multiple linear regression กรณี $d=2$ ที่ต้องการทำนาย `sepal_length` จาก `petal_width` และ `petal_length`




 โดยส่วนที่โค้ดนี้ ต่างจากโค้ดกรณี d=1 ที่ใช้ `petal_width` อย่างเดียวคือ แก้ `X` เป็น `X = iris[['petal_width','petal_length']]` เท่านั้น


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = iris[['petal_width','petal_length']]   ##### แก้แค่บรรทัดนี้เท่านั้น
Y = iris[['sepal_length']] # Y เป็น DataFrame
label = iris[['class']]    # label เป็น DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.6,
                                                    stratify=labels,
                                                    random_state=123)
print(f'Shape of training set: X_train {X_train.shape}, y_train {y_train.shape}')
print(f'Shape of test set:     X_test  {X_test.shape},  y_test  {y_test.shape}')

# สร้างและเทรนโมเดล
model = LinearRegression()
model.fit(X_train, y_train)

# ประสิทธิภาพกรณีทำนาย training set
y_train_pred = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)

# ประสิทธิภาพกรณีทำนาย test set
y_test_pred = model.predict(X_test)
mse_test  = mean_squared_error(y_test, y_test_pred)

print(f'MSE จากการทำนาย training set = {mse_train:.3f}') # เพิ่ม :.3f เข้าไปเพื่อพิมพ์ทศนิยมแค่ 3 ตำแหน่ง
print(f'MSE จากการทำนาย test set     = {mse_test:.3f}')

Shape of training set: X_train (90, 2), y_train (90, 1)
Shape of test set:     X_test  (60, 2),  y_test  (60, 1)
MSE จากการทำนาย training set = 0.153
MSE จากการทำนาย test set     = 0.170


### เลือกคุณลักษณะใดเป็นอินพุตดี

ไม่ใช่ว่าทุกคุณลักษณะที่เป็นอินพุตของโมเดล จะเป็นประโยชน์ต่อการทำนาย และในหลายกรณี คุณลักษณะที่ไม่เป็นประโยชน์กลับไปลดประสิทธิภาพของโมเดลอีกด้วย ดังนั้น จึงจำเป็นต้องเลือกคุณลักษณะที่เหมาะสม ซึ่งอาจพิจารณา ดังนี้

1. ขั้นแรกเลย ซึ่งควรทำเป็นอย่างแรก คือการอาศัยความรู้ (Domain knowledge) ในการตัดคุณลักษณะที่ไม่ดีออกไป
2. ถ้าจำนวนคุณลักษณะมีไม่มาก ก็อาจลองทดสอบทุกกลุ่มย่อยของคุณลักษณะที่เป็นไปได้ทั้งหมด แล้วเปรียบเทียบค่า MSE ของแต่ละแบบ การทำแบบนี้เรียกว่า **Exhaustive search** (การค้นหาแบบทั้งหมด) ซึ่งจริง ๆ เป็นวิธีการที่ไม่ดีนักในกรณีที่มีคุณลักษณะจำนวนมาก (ใช้เวลาในการคำนวณนาน หลายล้านปีก็ยังไม่เสร็จ)
3. ถ้าจำนวนคุณลักษณะมีมาก อาจใช้เทคนิคจำพวก Partial search ซึ่งไม่ขอกล่าวถึงในวิชานี้

สำหรับโค้ดในส่วนนี้ แสดงโค้ดของการทำ Exhaustive search เพื่อเปรียบเทียบค่า MSE ของ training set



In [ ]:
# โค้ดส่วนนี้เหมือนเดิม
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

iris_train, iris_test = train_test_split(iris,
                                       train_size=0.6,
                                       stratify=iris[['class']],
                                       random_state=None)

# รายการของชุดคุณลักษณะที่ต้องการใช้ โดยจะลองทุก combination
columns_list = [['petal_width'],
                ['sepal_width'],
                ['petal_length'],
                ['petal_width', 'sepal_width'],
                ['petal_width', 'petal_length'],
                ['sepal_width', 'petal_length'],
                ['petal_width', 'sepal_width', 'petal_length']]

# โค้ดเหมือนเดิม เพียงแต่ใช้ Loop ในการลองว่าชุดของคุณลักษณะที่เลือกแต่ละแบบ ให้ MSE เท่าไร
for columns in columns_list:
    X_train = iris_train[columns]
    y_train = iris_train['sepal_length']
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    print(f'{columns} --> MSE = {mse_train}')

['petal_width'] --> MSE = 0.22771068996238963
['sepal_width'] --> MSE = 0.6973324250513978
['petal_length'] --> MSE = 0.15742467134316898
['petal_width', 'sepal_width'] --> MSE = 0.19787011189038234
['petal_width', 'petal_length'] --> MSE = 0.14656578806674972
['sepal_width', 'petal_length'] --> MSE = 0.10092447622108516
['petal_width', 'sepal_width', 'petal_length'] --> MSE = 0.0783176388314411


จากผลข้างต้น จะเห็นว่า การใช้ 3 คุณลักษณะเป็นอินพุตของโมเดล จะให้ผลดีที่สุด อย่างไรก็ตาม สำหรับชุดข้อมูลอื่น ๆ ที่ไม่ใช่ดอกไอริส การใช้จำนวนคุณลักษณะมาก ๆ ไม่ได้แปลว่าจะให้ผลลัพธ์ที่ดีเสมอไป


## การ Standardize ข้อมูล หรือ Normalize ข้อมูล

ทุกท่านครับ Linear regression ที่เราทำมาแต่ต้น ดูไม่ได้มีปัญหาอะไรใช่ไหมครับ แต่ที่จริงแล้ว เราได้ข้ามขั้นตอนย่อยของ Data preparation/Data preprocessing ที่สำคัญอันหนึ่งคือ **Data normalization** หรือ **Data standardization**

ให้นึกถึงกรณีอื่น ๆ ที่ไม่ใช่ข้อมูลดอกไอริส โดยข้อมูลเหล่านั้น อาจมีค่าในแต่ละคอลัมน์เป็นคนละหน่วย เช่นคอลัมน์แรกอาจเป็นอายุ ซึ่งมีหน่วย ปี มีค่าในช่วง 0 - 100 ส่วนอีกคอลัมน์เป็นเงินเดือนที่มีหน่วยบาท ซึ่งอาจจะอยู่ในช่วง 10000-200000 บาท เป็นต้น การที่ค่ามีช่วงกว้างหรือแคบต่างกันแบบสุดโต่งเช่นนี้ มักจะมีผลเสียต่อการคำนวณหาค่าพารามิเตอร์ที่ดีที่สุดของโมเดล (ทำไม? ต้องเข้าใจกระบวนการหาหาค่าพารามิเตอร์ของโมเดลก่อน ซึ่งจะละไว้ก่อน)

การที่จะทำให้แต่ละคอลัมน์มีช่วงของค่าเท่า ๆ กัน ทำได้หลายแบบ ได้แก่
1. คำนวณค่าเฉลี่ย ($\mu$) และค่าส่วนเบี่ยงเบนมาตรฐาน ($\sigma$) ของคอลัมน์ ๆ หนึ่ง จากนั้นนำมาเข้าสูตรหา z-score
$$
\frac{x - \mu}{\sigma}
$$
โดย $x$ คือค่าแต่ละค่าในคอลัมน์นั้น การปรับช่วงของข้อมูลด้วยวิธีการนี้ เรียกว่า **Data standardization** (แต่จะเรียกว่า Data normalization ก็ไม่ผิด)

2. ปรับสเกลของข้อมูลให้ตกอยู่ในช่วงที่ต้องการ เช่น -1 ถึง 1 หรือ 0 ถึง 1 เป็นต้น เรียกว่า Data normalization

ในส่วนต่อไปนี้ จะแสดงวิธีการทำ Standardization โดยใช้ Pandas และจากแสดงว่า Scikit-learn ก็สามารถทำ Standardization ได้เช่นกัน

อย่าลืมว่า หากพิจารณา DataFrame `iris` ทั้ง 150 ดอก เราสามารถ Standardization ข้อมูลรวดเดียวทั้ง 150 ดอกได้ ซึ่งก็สะดวกดี (สามารถทำได้) แต่เพื่อที่จะเป็นตัวอย่าง เราจะไม่ทำอย่างนั้น เราจะพิจารณา training set และ test set แยกจากกัน และจะ Standardization เฉพาะคุณลักษณะที่เป็นอินพุตของโมเดลเท่านั้น ไม่ Standardization ค่าที่ต้องการทำนาย

> **Note:** จะแสดงเฉพาะการทำ Standardization ส่วนการ Normalize ให้อยู่ในช่วงที่ต้องการ เช่น 0 ถึง 1 จะให้ฝึกทำเป็นการบ้าน

In [ ]:
from sklearn.model_selection import train_test_split

X = iris[['petal_width','petal_length','sepal_width']]  # X เป็น DataFrame
Y = iris[['sepal_length']] # Y เป็น DataFrame
label = iris[['class']]    # label เป็น DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.6,
                                                    stratify=labels,
                                                    random_state=123)

### Standardization ด้วย Pandas

ในตัวอย่างต่อจากนี้ จะใช้คำสั่ง `df.mean()` และ `df.std(ddof=0)` ในการคำนวณค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐาน โดยค่า SD จะเป็นค่าของประชากร (Population SD)

> **Note:** ในทางปฏิบัติ จะใช้เพียง `df.std()` ที่ให้ค่า SD ของกลุ่มตัวอย่าง (Sample SD) ก็ได้ เพราะเป้าหมายของการทำ Normalization นั้น ทำด้วยเหตุผลเพียงแค่ไม่ให้ข้อมูลกระจายตัวมากเกินไปในมิติใดมิติหนึ่ง
>ซึ่งสาเหตุที่จะใช้ `df.std(ddof=0)` ก็เพียงเพื่อให้ผลลัพธ์จากการคำนวณด้วย Pandas ตรงกับของ Scikit-learn เท่านั้น



In [ ]:
mu = X_train.mean()
sd = X_train.std(ddof=0)  # ถ้าใช้ ddof=0 แล้วจะได้ผล Standardization ตรงกับ Scikit-learn
print(f'mean = {mu.values}\nsd = {sd.values}')

mean = [1.19111111 3.72888889 3.02333333]
sd = [0.75345787 1.70099608 0.43231933]


**แบบที่ 1**

In [ ]:
(X_train - mu)/sd

,petal_width,petal_length,sepal_width
24,-1.315417,-1.075187,0.871270
147,1.073569,0.864853,-0.053972
88,0.144519,0.218173,-0.053972
123,0.808126,0.688485,-0.747904
31,-1.049974,-1.310343,0.871270
...,...,...,...
52,0.409962,0.688485,0.177338
41,-1.182695,-1.427922,-1.673146
12,-1.448138,-1.369132,-0.053972
20,-1.315417,-1.192765,0.871270


ทำไมโค้ดข้างต้นเอา `X_train` ซึ่งเป็น DataFrame มีมิติ `(90,3)` มาลบกับ `X_train.mean()` และหารด้วย `X_train.std()` ซึ่งเป็น Series ที่มีมิติแค่ `(3,)` ได้ล่ะ?

In [ ]:
print(f'shape of X_train = {X_train.shape}')
print(f'shape of X_train.mean() = {mu.shape}')
print(f'shape of X_train.std() = {sd.shape}')

shape of X_train = (90, 3)
shape of X_train.mean() = (3,)
shape of X_train.std() = (3,)


ก็เพราะว่า `X_train.mean()` และ `X_train.std()` มี index ของ Series คือ `petal_length`, `petal_width` และ `sepal_width` ทำให้ Pandas เอามาลบได้แต่ละคอลัมน์ได้ถูกคอลัมน์

หลังจาก Normalize `X_train` แล้ว เราก็สามารถ Normalize `X_test` ได้ด้วย mean และ SD ของ `X_train`

```python
(X_test - mu)/sd
```

> **คำถาม**
> - ทำไมต้อง Normalize โดยใช้ mean และ sd ของ `X_train`? ทำไม `X_test` ไม่ใช้ mean และ sd ของมันเอง?
> - ทำไมข้อมูล `X_train` ที่ Standardize แล้วมีค่าเฉลี่ย = 0 และ SD = 1

**แบบที่ 2** ใช้คำสั่ง `df.apply()`

คำสั่ง `apply()` เป็นคำสั่งที่ทำให้เราสามารถเรียกใช้ฟังก์ชันที่เราสร้างขึ้นเอง มาคำนวณแต่ละคอลัมน์ของ `df` แยกจากกัน

ว่าแต่นักศึกษายังจำวิธีสร้างฟังก์ชันในภาษา Python ได้ไหม? ถ้าพอนึกออก มาดูตัวอย่างการใช้ `df.apply()` กันเลย

In [ ]:
# นิยามฟังก์ชันที่เองชื่อ normalize
def normalize(col):
  # ตัวแปรชื่อ col ที่รับมาจาก apply จะเป็นเพียงคอลัมน์ ๆ หนึ่งของ X_train
  return (col - col.mean())/col.std(ddof=0) # Standardization

# apply จะส่งทีละคอลัมน์ของ X_train ให้ฟังก์ชัน normalize
# และหลังจากที่ฟังก์ชัน normalize คำนวณเสร็จ ก็จะถูกรวบรวมกลับเป็น DataFrame
X_train.apply(normalize)

,petal_width,petal_length,sepal_width
24,-1.315417,-1.075187,0.871270
147,1.073569,0.864853,-0.053972
88,0.144519,0.218173,-0.053972
123,0.808126,0.688485,-0.747904
31,-1.049974,-1.310343,0.871270
...,...,...,...
52,0.409962,0.688485,0.177338
41,-1.182695,-1.427922,-1.673146
12,-1.448138,-1.369132,-0.053972
20,-1.315417,-1.192765,0.871270


### Standardization ด้วย Scikit-learn

ก่อนอื่นต้องนำเข้า `StandardScaler()` เข้าก่อน และ สร้างตัวแปรที่จะเป็นตัวกลางในการทำ Standardization (สมมติชื่อ `scaler`)
```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
```

การนำไปใช้ ต้องนำ `scaler` ไปคำนวณค่า mean และ sd ของข้อมูลที่ต้องการก่อน ด้วยคำสั่ง
```python
scaler.fit(ข้อมูล)
```

<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-transformer.fit.svg" width=30%> 
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

ซึ่งเมื่อทำแล้ว ค่า mean และ sd ของข้อมูล สามารถเข้าถึงได้จาก `scaler.mean_` และ `scaler.scale_`
จากนั้นหากต้องการ Standardize ข้อมูลใด ๆ ก็เรียกใช้
```python
ข้อมูลที่แปลงแล้ว = scaler.transform(ข้อมูล)
```

<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-transformer.transform.svg" width=50%> 
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

หรือในกรณีที่ต้องการทำทั้ง fit และ transform ในคราวเดียว ไม่ต้องเสียเวลาเรียก `.fit()` แล้วค่อย `.transform()` ก็สามารถทำได้ ดังนี้
```python
ข้อมูลที่แปลงแล้ว = scaler.fit_transform(ข้อมูล)
```

<figure>
<img src="https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/figures/api_diagram-transformer.fit_transform.svg" width=50%>
<figcaption>ที่มารูป scikit-learn MOOC by scikit-learn developers (https://inria.github.io/scikit-learn-mooc/)</figcaption>
</figure>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# คำนวณ mean และ sd ของ X_train
scaler.fit(X_train)

# ทำ Standardization กับ X_train
new_X_train = scaler.transform(X_train)

# ทำ Standardization กับ X_test ด้วย mean และ sd ของ X_train
new_X_test = scaler.transform(X_test)

# ลองพิมพ์ผลลัพธ์การทำ Standardization
mu = scaler.mean_
sd = scaler.scale_
print(f'mean = {mu}\nsd = {sd}')

mean = [1.19111111 3.72888889 3.02333333]
sd = [0.75345787 1.70099608 0.43231933]


ทั้งนี้ ตัวแปร `new_X_train` และ `new_X_test` ที่ได้จาก `scaler.transform()` จะไม่ใช่เป็น DataFrame แต่เป็น Numpy array ซึ่งจะสูญเสียชื่อคอลัมน์ไป ดังนั้น หากต้องการทำให้เป็น DataFrame ก็สามารถทำได้ ดังนี้ ซึ่งเมื่อแสดงผลออกมา จะพบว่า ผลของการ Standardize ตรงกับผลจากที่คำนวณเองด้วย Pandas

In [ ]:
# แสดงตัวอย่างสำหรับ new_X_train ซึ่ง new_X_test ก็ทำในลักษณะเดียวกัน
new_X_train = pd.DataFrame(new_X_train,
             columns=X_train.columns,  # เอาชื่อคอลัมน์มาจากต้นฉบับ
             index=X_train.index)      # เอาชื่อแถวมาจากต้นฉบับ
new_X_train

,petal_width,petal_length,sepal_width
24,-1.315417,-1.075187,0.871270
147,1.073569,0.864853,-0.053972
88,0.144519,0.218173,-0.053972
123,0.808126,0.688485,-0.747904
31,-1.049974,-1.310343,0.871270
...,...,...,...
52,0.409962,0.688485,0.177338
41,-1.182695,-1.427922,-1.673146
12,-1.448138,-1.369132,-0.053972
20,-1.315417,-1.192765,0.871270


### (ศึกษาเอง) กราฟก่อนและหลังการทำ Standardization

ทีนี้ ลองวาดกราฟเปรียบเทียบก่อนและหลังการ Standardize ซึ่งจะยกตัวอย่างคู่ `petal_width` กับ `petal_length` โดยจะทดลองวาดเป็นกราฟ 2 กราฟแยกกันโดยใช้ `plotly.graph_objects` ร่วมกับ `plotly.subplots` ซึ่งจากกราฟ จะเห็นว่า ก่อนและหลังเป็นรูปเดียวกัน เพียงแต่ค่าพิกัดแกนนอนและแกนตั้งต่างกัน

> **Note:** สำหรับคำสั่งในการวาดกราฟ ให้นักศึกษา**ลองศึกษาด้วยตนเอง** ตามลิงค์นี้ https://plotly.com/python/subplots/ แต่โดยหลักการแล้ว จะใช้คำสั่ง `make_subplots()` เพื่อกำหนดจำนวนช่องที่ต้องการวาดกราฟก่อน จากนั้นจึงใส่กราฟเข้าไปทีละช่องด้วยการระบุว่าจะใส่แถวหรือคอลัมน์ที่เท่าไร


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)  # สร้างช่องไว้ล่วงหน้า 2 ช่อง เพื่อรอใส่กราฟ

obj_1 = go.Scatter(x=X_train['petal_width'],
                   y=X_train['petal_length'],
                   mode='markers',
                   name='ก่อนทำ')
obj_2 = go.Scatter(x=new_X_train['petal_width'],
                   y=new_X_train['petal_length'],
                   mode='markers',
                   name='หลังทำ')

fig.add_trace(obj_1, row=1, col=1) # ใส่กราฟลงช่องแรก
fig.add_trace(obj_2, row=1, col=2) # ใส่กราฟลงช่องที่สอง

# แก้ไขชื่อแกนนอนและแกนตั้ง
fig.update_xaxes(title_text='petal_width', row=1, col=1)
fig.update_yaxes(title_text='petal_length', row=1, col=1)

fig.update_xaxes(title_text='petal_width', row=1, col=2)
fig.update_yaxes(title_text='petal_length', row=1, col=2)

# แก้ไขขนาดและชื่อกราฟ
fig.update_layout(height=400, width=1000, title_text="แผนภาพเปรียบเทียบก่อนและหลัง Standardization")
fig.show()

### ลองทำ: Normalization ให้อยู่ในช่วง -1 ถึง 1

การทำ Normalization ให้ข้อมูลอยู่ในช่วง -1 ถึง 1 นั้นสามารถทำได้ด้วย Pandas ได้ โดยนักศึกษาอาจลองทำด้วยตนเองดูก่อน (ซึ่งจริง ๆ ไม่ยาก) แต่สำหรับในส่วนนี้ อยากจะลองให้ทำโดย `MinMaxScaler` ที่เป็นฟังก์ชันสำเร็จรูปของ Scikit-learn ซึ่งจะง่ายกว่า โดยสามารถนำเข้าได้ดังนี้

```python
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1,1))  # ช่วง -1 ถึง 1
```
เมื่อสร้าง `scaler` ขึ้นมาแล้ว สามารถเรียกใช้งานได้ในแบบเดียวกับ `StandardScaler` ดังนั้น ขอให้นักศึกษาลองใช้งาน `scaler` ตัวนี้ด้วยตนเองในการ Normalize หากต้องการแหล่งอ้างอิง อาจดูคำอธิบายเพิ่มเติมจาก [ที่นี่](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)


### ลองทำ: ให้นำ `StandardScaler()` ไปใช้ก่อนการเทรนโมเดลและการทำนาย

```python
X = iris[['petal_width','petal_length']]  # X เป็น DataFrame
Y = iris[['sepal_length']] # Y เป็น DataFrame
label = iris[['class']]    # label เป็น DataFrame

### เติมโค้ดด้านล่างให้สมบูรณ์ ###
X_train, X_test, Y_train, Y_test = train_test_split(_____)

# Standardization
scaler = StandardScaler()
new_X_train = scaler.fit_transform(X_train)
new_X_test = scaler.transform(X_test)

# เทรนโมเดล
model = ______
model.fit(_____)

# ประสิทธิภาพกรณีทำนาย training set
model.predict(_____)
mean_squared_error(_____)

# ประสิทธิภาพกรณีทำนาย test set
model.predict(_____)
mean_squared_error(_____)
```

## (ศึกษาเอง) ค่าความผิดพลาดอื่น ๆ

นักศึกษาทราบแล้วว่า สามารถวัดความผิดพลาดของการทำนายได้โดยใช้ Mean Squared Error (MSE) แต่ยังมีค่าอื่น ๆ อีก สรุปรวมได้ดังตารางต่อไปนี้ โดย `y_true` คือค่าที่แท้จริง และ `y_pred` คือค่าที่ทำนายได้

| ค่า | สูตรคณิตศาสตร์&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |  `from sklearn.metrics import ______`  |
|:-- |:-- |:-- |
| [Mean Squared Error (MSE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) | $\frac{1}{n} \sum_{i=1}^n (\hat{y}^{(i)} - y^{(i)})^2$ | `mean_squared_error(y_true, y_pred)` |
| [Root Mean Squared Error (RMSE)](https://scikit-learn.org/dev/modules/generated/sklearn.metrics.root_mean_squared_error.html) | $\sqrt{MSE}$ | `root_mean_squared_error(y_true, y_pred)` |
| [Mean Absolute Error (MAE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) | $\frac{1}{n} \sum_{i=1}^n \| \hat{y}^{(i)} - y^{(i)}\|$ |  `mean_absolute_error(y_true, y_pred)` |

ค่าอื่น ๆ ที่สำคัญไม่แพ้กัน ได้แก่ค่า R-Squared หรือ coefficient of determination ซึ่งขอไม่กล่าวถึงรายละเอียด

**ลองทำ:** ให้ลองคำนวณ MAE
```python
from sklearn.metrics import mean_absolute_error
____________
```

> **Note:** สำหรับคำสั่ง `root_mean_squared_error` จะใช้ได้ใน Scikit-learn เวอร์ชัน 1.4 ขึ้นไป สำหรับการตรวจสอบเวอร์ชัน Scikit-learn ที่ลงมาให้ใน Google Colab ตรวจสอบได้จาก
> ```python
> import sklearn
> sklearn.__version__
> ```

## สรุปภาพรวมการพัฒนาโมเดล

กระบวนการทั้งหมดในการทำนายค่า `sepal_length` เป็นดังนี้
1. ตั้งโจทย์ปัญหาว่าจะทำนายอะไร (Problem formulation) ซึ่งกรณีนี้คือต้องการทำนาย sepal length  และตั้งคำถามว่า จะใช้อะไรเป็นตัวแปรต้นในการทำนาย ซึ่งต้องอาศัยความรู้เฉพาะทาง (Domain expertise) ว่ามันมีความสัมพันธ์กัน
2. เดินสายเก็บข้อมูล (Data collection) วัดความยาวและความกว้างของดอกไอริส โดยในกรณีนี้มีคนเก็บข้อมูลไว้ให้แล้ว
3. ทำความเข้าใจข้อมูล (Data understanding) ซึ่งทำไปในคาบก่อนที่ทำ Exploratory Data Analysis
4. จัดการข้อมูลเบื้องต้นก่อนการเทรนโมเดล (Data preprocessing) ได้แก่ การแบ่งข้อมูล train/test และการทำ Standardization
5. การเลือกโมเดล ซึ่งกรณีนี้ เราเลือกโมเดล Linear regression เพราะเห็นความสัมพันธ์เชิงเส้นในข้อมูล
6. เทรนโมเดล (Model training)
7. ใช้โมเดลทำนาย (Prediction) และทดสอบโมเดล (Evaluation) ด้วย MSE

คำถามที่มักจะพบบ่อยเกี่ยวกับการใช้โมเดล Linear regression ก็คือ ตัวโมเดล Linear regression เหมาะสมกับชุดข้อมูลที่มีอยู่หรือไม่ โดยประเด็นนี้ จะเป็นประเด็นที่กล่าวถึงในหัวข้อต่อไป
